In [1]:
import pandas as pd
import time
import datetime
import numpy as np

In [2]:
# 属性名
columns_names = ['Flight_number', 'Leg 'Origin', 'Destination','Depart_time', 'Arrive_time', 'Flight_time', 'B_class_sales',
                 'G_class_sales', 'H_class_sales', 'K_class_sales', 'L_class_sales', 'M_class_sales', 'Q_class_sales','S_class_sales', 'T_class_sales',
                 'V_class_sales', 'Y_class_sales']

In [3]:
main_df = pd.DataFrame(columns=columns_names)

In [4]:
# 读入原始文件
products_df = pd.read_csv(r'../INPUT/data_fam_products.csv')
schedule_df = pd.read_csv(r'../INPUT/data_fam_schedule.csv')

In [5]:
# 填充航班号、起飞机场、到达机场、航段号
main_df['Flight_number'] = schedule_df['flight']
main_df['Origin'] = schedule_df['origin']
main_df['Destination'] = schedule_df['destination']
main_df['Leg'] = main_df['Leg'] + main_df['Origin'] + main_df['Destination']

In [6]:
# 初始值
main_df[[ 'B_class_sales','G_class_sales', 'H_class_sales', 'K_class_sales', 'L_class_sales', 'M_class_sales',
          'Q_class_sales','S_class_sales', 'T_class_sales','V_class_sales', 'Y_class_sales']] =0

In [7]:
# 将时间转换为UTC时间
main_df['Depart_time'] = schedule_df['deptime'] - schedule_df['depoff']
main_df['Arrive_time'] = schedule_df['arrtime'] - schedule_df['arroff']

for index in range(815):  # 修正跨日期飞行时间
    if main_df.loc[index, 'Depart_time'] >= 2400:
        main_df.loc[index, 'Depart_time'] -= 2400
    if main_df.loc[index, 'Arrive_time'] >= 2400:
        main_df.loc[index, 'Arrive_time'] -= 2400

In [8]:
# 将起飞、到达时间格式改为xxxx，数据类型为str
for index in range(len(main_df)):
    arrive_time = str(main_df.loc[index, 'Arrive_time'])  # 到达时间
    if len(arrive_time) < 4:  # 时间格式为xxxx
        arrive_time = '0' * (4 - len(arrive_time)) + arrive_time  # 添加缺失的0
    main_df.loc[index, 'Arrive_time'] = arrive_time
    
    depart_time = str(main_df.loc[index, 'Depart_time'])  # 起飞时间
    if len(depart_time) < 4:
        depart_time = '0' * (4 - len(depart_time)) + depart_time
    main_df.loc[index, 'Depart_time'] = depart_time

In [9]:
# 格式转换函数
def getTimefmt(self):  
    if len(str(self))  < 4:
        self = '0' * (4 - len(str(self))) + str(self)
    temptime = datetime.datetime.strptime((str(self)[0:2])+":"+(str(self)[2:]),'%H:%M')
    return temptime

In [10]:
# 创建副本
temp_df = main_df.copy()
temp_df['Depart_time'] = temp_df['Depart_time'].apply(getTimefmt)
temp_df['Arrive_time'] = temp_df['Arrive_time'].apply(getTimefmt)

In [11]:
# 计算并修正飞行时间
main_df['Flight_time'] = (temp_df['Arrive_time'] - temp_df['Depart_time']).values/np.timedelta64(1, 'h')
for index in range(len(main_df)):
    if main_df.loc[index, 'Flight_time'] < 0:
        main_df.loc[index, 'Flight_time'] += 24
main_df

,Flight_number,Flight_segment,Origin,Destination,Depart_time,Arrive_time,Flight_time,B_class_sales,G_class_sales,H_class_sales,K_class_sales,L_class_sales,M_class_sales,Q_class_sales,S_class_sales,T_class_sales,V_class_sales,Y_class_sales
0,AA0001,AA0001EDBTFB,EDB,TFB,1155,1738,5.716667,0,0,0,0,0,0,0,0,0,0,0
1,AA0002,AA0002TFBEDB,TFB,EDB,2105,0159,4.900000,0,0,0,0,0,0,0,0,0,0,0
2,AA0003,AA0003EDBTFB,EDB,TFB,2140,0325,5.750000,0,0,0,0,0,0,0,0,0,0,0
3,AA0004,AA0004TFBEDB,TFB,EDB,1540,2040,5.000000,0,0,0,0,0,0,0,0,0,0,0
4,AA0005,AA0005EDBMBY,EDB,MBY,1310,1902,5.866667,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
810,BA2645,BA2645QEYTNG,QEY,TNG,0005,0150,1.750000,0,0,0,0,0,0,0,0,0,0,0
811,BA2670,BA2670TFBZMX,TFB,ZMX,2035,2135,1.000000,0,0,0,0,0,0,0,0,0,0,0
812,BA2700,BA2700NGSMBY,NGS,MBY,0316,0530,2.233333,0,0,0,0,0,0,0,0,0,0,0
813,AA2701,AA2701BODMBT,BOD,MBT,1445,1943,4.966667,0,0,0,0,0,0,0,0,0,0,0


In [13]:
read_date = products_df.columns.tolist()[7:]  # 取出记录点df
read_date_df = products_df[read_date]

In [14]:
products_df['Sales'] = read_date_df.apply(lambda x:x.sum(), axis=1)  # 对所有计算每一种产品的销售量
products_df['Sales']

0         2.290000
1         2.070000
2         0.660000
3         0.000000
4        13.120000
           ...    
47185     8.666667
47186     0.000000
47187     1.430000
47188    14.603333
47189    27.300000
Name: Sales, Length: 47190, dtype: float64

In [15]:
'''
计算各航段各舱位的销售量：

在products表中获取航段号（12位）
通过航段号定位主表中的索引
使用索引计算各舱位的销售量
'''
for index in range(len(products_df)):
    leg = products_df.loc[index, 'Flight1']  # 获取航段一号码
    class_name = products_df.loc[index, 'Class'] + '_class_sales'  # 获取舱位等级
    target_index = (main_df['Leg']== leg)  # 获取主表索引
    main_df.loc[target_index, class_name] += products_df.loc[index, 'Sales']

In [17]:
print(products_df.loc[1, 'Flight2'])  # 无航段二、三的字符

.


In [16]:
for index in range(len(products_df)):
    leg = products_df.loc[index, 'Flight2']  # 获取航段二号码
    if leg == '.':
        continue
    class_name = products_df.loc[index, 'Class'] + '_class_sales'  # 获取舱位等级
    target_index = (main_df['Leg']== leg)  # 获取主表索引
    main_df.loc[target_index, class_name] += products_df.loc[index, 'Sales']

In [17]:
for index in range(len(products_df)):
    leg = products_df.loc[index, 'Flight2']  # 获取航段三号码
    if leg == '.':
        continue
    class_name = products_df.loc[index, 'Class'] + '_class_sales'  # 获取舱位等级
    target_index = (main_df['Leg']== leg)  # 获取主表索引
    main_df.loc[target_index, class_name] += products_df.loc[index, 'Sales']

In [18]:
# 计算各航段的总销售量
main_df['Total_sales'] = 0

for index in range(len(main_df)):
    main_df.loc[index, 'Total_sales']=main_df.iloc[index, 7:].sum()

In [19]:
main_df

,Flight_number,Flight_segment,Origin,Destination,Depart_time,Arrive_time,Flight_time,B_class_sales,G_class_sales,H_class_sales,K_class_sales,L_class_sales,M_class_sales,Q_class_sales,S_class_sales,T_class_sales,V_class_sales,Y_class_sales,Total_sales
0,AA0001,AA0001EDBTFB,EDB,TFB,1155,1738,5.716667,8.03,4.43,8.96,10.46,70.5,11.51,6.54,3.37,12.67,32.38,66.45,235.30
1,AA0002,AA0002TFBEDB,TFB,EDB,2105,0159,4.900000,13.37,3.8,24.52,13.21,83.25,7.83,11.98,1.21,0.95,52.75,27.92,240.79
2,AA0003,AA0003EDBTFB,EDB,TFB,2140,0325,5.750000,4.33,5.63,17.01,13.79,15.31,8.24,28.76,0.7,2.97,21.98,27.33,146.05
3,AA0004,AA0004TFBEDB,TFB,EDB,1540,2040,5.000000,54.9,3.44,4.01,3.44,36.5,13.31,13.5,22.43,3.44,17.57,85.51,258.05
4,AA0005,AA0005EDBMBY,EDB,MBY,1310,1902,5.866667,0.03,59.57,11.41,11.52,13.69,0.69,11.01,0.0,35.34,46.76,7.49,197.51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
810,BA2645,BA2645QEYTNG,QEY,TNG,0005,0150,1.750000,7.2,17.92,0.4,4.77,8.09,0.0,2.23,1.94,30.97,8.37,2.24,84.13
811,BA2670,BA2670TFBZMX,TFB,ZMX,2035,2135,1.000000,2.75,28.7,0.92,15.99,17.11,4.37,14.94,6.78,5.4,13.94,6.2,117.10
812,BA2700,BA2700NGSMBY,NGS,MBY,0316,0530,2.233333,9.923333,8.97,2.86,0.0,56.853333,15.903333,8.666667,0.0,1.43,14.603333,27.3,146.51
813,AA2701,AA2701BODMBT,BOD,MBT,1445,1943,4.966667,9.923333,8.97,2.86,0.0,56.853333,15.903333,8.666667,0.0,1.43,14.603333,27.3,146.51


In [20]:
main_df.to_csv('main_table.csv')